In [15]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load the datasets
cems_data = pd.read_csv('../data/raw/point/2022hc_cb6_22m/inputs/ptegu/egu_cems_2022_POINT_20240615_2022cems_stackfix2_23jul2024_v0.csv', 
                        delimiter=',', quotechar='"', skiprows=6)
nei_data = pd.read_csv('../data/raw/2021_NEI_Facility_summary.csv', delimiter=',', quotechar='"')

/var/folders/v9/zhvyqdfn1n7d3v6cwvmxqh7c0000gn/T/ipykernel_8283/3009881509.py:7: DtypeWarning: Columns (7,8,10,29,32,41,50) have mixed types. Specify dtype option on import or set low_memory=False.
  cems_data = pd.read_csv('../data/raw/point/2022hc_cb6_22m/inputs/ptegu/egu_cems_2022_POINT_20240615_2022cems_stackfix2_23jul2024_v0.csv',
/var/folders/v9/zhvyqdfn1n7d3v6cwvmxqh7c0000gn/T/ipykernel_8283/3009881509.py:9: DtypeWarning: Columns (2,7,8,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  nei_data = pd.read_csv('../data/raw/2021_NEI_Facility_summary.csv', delimiter=',', quotechar='"')


In [7]:
# Create a unique list of facilities in CEMS data
cems_facilities = cems_data[['facility_id', 'oris_facility_code', 'facility_name', 'latitude', 'longitude']].drop_duplicates()

# If 'oris_facility_code' exists in NEI data, use direct matching
if 'oris_facility_code' in nei_data.columns:
    nei_filtered = nei_data[nei_data['oris_facility_code'].isin(cems_facilities['oris_facility_code'])]
    print(f"Matched {len(nei_filtered)} facilities using ORIS code")
# Otherwise, try location-based matching
else:
    print("No direct ORIS code match possible, using location-based matching...")
    
    # Function to find closest facility by coordinates
    def find_closest_facility(lat, lon, reference_df, max_distance=0.01):
        """Find the closest facility within a small distance threshold"""
        distances = ((reference_df['site latitude'] - lat)**2 + 
                     (reference_df['site longitude'] - lon)**2)**0.5
        closest_idx = distances.idxmin()
        min_distance = distances.min()
        
        if min_distance <= max_distance:
            return reference_df.iloc[closest_idx]['eis facility id']
        return None
    
    # Add matched facility ID to CEMS data
    cems_facilities['matched_nei_id'] = cems_facilities.apply(
        lambda row: find_closest_facility(row['latitude'], row['longitude'], nei_data),
        axis=1
    )
    
    # Filter NEI data to only include matched facilities
    matched_ids = cems_facilities['matched_nei_id'].dropna().unique()
    nei_filtered = nei_data[nei_data['eis facility id'].isin(matched_ids)]
    
    print(f"Matched {len(matched_ids)} facilities using location-based matching")

# If few matches, try name-based fuzzy matching as a backup
if len(nei_filtered) < 0.5 * len(cems_facilities):
    print("Few matches found, attempting name-based fuzzy matching...")
    
    # Get unique facility names from both datasets
    cems_names = cems_facilities['facility_name'].unique()
    nei_names = nei_data['site name'].unique()
    
    # Create mapping dictionary using fuzzy matching
    name_matches = {}
    for cems_name in cems_names:
        match, score = process.extractOne(cems_name, nei_names)
        if score >= 80:  # Threshold for considering a match
            name_matches[cems_name] = match
    
    # Add matched names to the filter
    name_matched_facilities = nei_data[nei_data['site name'].isin(name_matches.values())]
    nei_filtered = pd.concat([nei_filtered, name_matched_facilities]).drop_duplicates()
    
    print(f"Added {len(name_matched_facilities)} facilities using name-based matching")

# Final filtered NEI dataset
print(f"Final NEI dataset contains {len(nei_filtered)} unique facilities")

No direct ORIS code match possible, using location-based matching...
Matched 1129 facilities using location-based matching
Final NEI dataset contains 47453 unique facilities


In [16]:
nei_data.columns.unique()

Index(['state', 'fips state code', 'tribal name', 'fips code', 'county',
       'eis facility id', 'program system code', 'agency facility id',
       'tri facility id', 'company name', 'site name', 'primary naics code',
       'primary naics description', 'facility source type', 'site latitude',
       'site longitude', 'address', 'city', 'zip code', 'postal abbreviation',
       'reporting period', 'emissions operating type', 'pollutant code',
       'pollutant desc', 'pollutant type(s)', 'hap type', 'total emissions',
       'emissions uom', 'data set', 'outlier minimum', 'outlier maximum',
       'outlier?', 'national maximum'],
      dtype='object')

In [18]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load the datasets
cems_data = pd.read_csv('../data/raw/point/2022hc_cb6_22m/inputs/ptegu/egu_cems_2022_POINT_20240615_2022cems_stackfix2_23jul2024_v0.csv', 
                        delimiter=',', quotechar='"', skiprows=6)
nei_data = pd.read_csv('../data/raw/2021_NEI_Facility_summary.csv', delimiter=',', quotechar='"')

# Define EGU NAICS codes
egu_naics = ['2211', '221111', '221112', '221113', '221114', '221115', '221116', '221117', '221118']

# Filter NEI data for facilities with EGU NAICS codes
# Convert NAICS code to string first to handle NaN and numeric formats
nei_data['primary naics code'] = nei_data['primary naics code'].astype(str)

# Create a filter mask for rows where NAICS starts with any of the EGU codes
naics_filter = nei_data['primary naics code'].apply(lambda x: any(x.startswith(code) for code in egu_naics))

# Apply the filter
nei_egu_data = nei_data[naics_filter]

print(f"Total NEI facilities: {len(nei_data)}")
print(f"NEI EGU facilities: {len(nei_egu_data)}")

/var/folders/v9/zhvyqdfn1n7d3v6cwvmxqh7c0000gn/T/ipykernel_8283/137357643.py:7: DtypeWarning: Columns (7,8,10,29,32,41,50) have mixed types. Specify dtype option on import or set low_memory=False.
  cems_data = pd.read_csv('../data/raw/point/2022hc_cb6_22m/inputs/ptegu/egu_cems_2022_POINT_20240615_2022cems_stackfix2_23jul2024_v0.csv',
/var/folders/v9/zhvyqdfn1n7d3v6cwvmxqh7c0000gn/T/ipykernel_8283/137357643.py:9: DtypeWarning: Columns (2,7,8,13,18) have mixed types. Specify dtype option on import or set low_memory=False.
  nei_data = pd.read_csv('../data/raw/2021_NEI_Facility_summary.csv', delimiter=',', quotechar='"')


Total NEI facilities: 2005169
NEI EGU facilities: 91922


In [22]:
nei_egu_data

,state,fips state code,tribal name,fips code,county,eis facility id,program system code,agency facility id,tri facility id,company name,...,pollutant desc,pollutant type(s),hap type,total emissions,emissions uom,data set,outlier minimum,outlier maximum,outlier?,national maximum
1306,CA,6.0,NaN,6031,Kings,17611,CARB,1614304140.0,93230HNFRD10596,"MRP SAN JOAQUIN ENERGY, LLC, HANFORD ENERGY PA...",...,Ethyl Benzene,HAP,HAP-VOC,6.922880,LB,2021NEI_FinalV2,NaN,NaN,No,5000.0
1307,CA,6.0,NaN,6031,Kings,17611,CARB,1614304140.0,93230HNFRD10596,"MRP SAN JOAQUIN ENERGY, LLC, HANFORD ENERGY PA...",...,"1,3-Butadiene",HAP,HAP-VOC,0.093000,LB,2021NEI_FinalV2,NaN,NaN,No,10000.0
1308,CA,6.0,NaN,6031,Kings,17611,CARB,1614304140.0,93230HNFRD10596,"MRP SAN JOAQUIN ENERGY, LLC, HANFORD ENERGY PA...",...,Acrolein,HAP,HAP-VOC,1.384576,LB,2021NEI_FinalV2,NaN,NaN,No,10000.0
1309,CA,6.0,NaN,6031,Kings,17611,CARB,1614304140.0,93230HNFRD10596,"MRP SAN JOAQUIN ENERGY, LLC, HANFORD ENERGY PA...",...,Toluene,HAP,HAP-VOC,28.124200,LB,2021NEI_FinalV2,NaN,NaN,No,200000.0
1310,CA,6.0,NaN,6031,Kings,17611,CARB,1614304140.0,93230HNFRD10596,"MRP SAN JOAQUIN ENERGY, LLC, HANFORD ENERGY PA...",...,Xylenes (Mixed Isomers),HAP,HAP-VOC,13.845760,LB,2021NEI_FinalV2,NaN,NaN,No,50000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004617,NC,37.0,NaN,37163,Sampson,21471211,NCDAQ,3716300158,NaN,"Prestage AgEnergy Operations, LLC",...,PM2.5 Primary (Filt + Cond),CAP,NaN,12.690000,TON,2021NEI_FinalV2,NaN,NaN,No,500.0
2004618,NC,37.0,NaN,37163,Sampson,21471211,NCDAQ,3716300158,NaN,"Prestage AgEnergy Operations, LLC",...,Remaining PMFINE portion of PM2.5-PRI,Other,NaN,6.934340,TON,2021NEI_FinalV2,NaN,NaN,No,NaN
2004619,NC,37.0,NaN,37163,Sampson,21471211,NCDAQ,3716300158,NaN,"Prestage AgEnergy Operations, LLC",...,Sulfur Dioxide,CAP,NaN,61.080000,TON,2021NEI_FinalV2,NaN,NaN,No,1000.0
2004620,NC,37.0,NaN,37163,Sampson,21471211,NCDAQ,3716300158,NaN,"Prestage AgEnergy Operations, LLC",...,Sulfate Portion of PM2.5-PRI,Other,NaN,0.828657,TON,2021NEI_FinalV2,NaN,NaN,No,NaN
